In [8]:
import os
# print(os.getcwd())
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
import pickle
import sklearn
print(sklearn.__version__)
import pandas
import numpy as np
import sys


from evaluation.metric import computeAccuracy, computeF1Score, computeAUC
# from sklearn.metrics import roc_auc_score

# %load_ext autoreload
# %autoreload
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import uniform
# %reload_ext pipelines.alcohol.AlcoholPipeline
from pipelines.alcohol import AlcoholPipeline

0.17


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# clf_alc_initial = pickle.load(open('./alcohol classifiers/clf_alc_UPDATED.p', 'rb'))
clf_fpa_initial = pickle.load(open('./alcohol classifiers/clf_fpa_UPDATED.p', 'rb'))
# clf_fpl_dl = pickle.load(open('./alcohol classifiers/clf_fpl_double_labeled', 'rb'))
df = pandas.read_csv("./data/alcohol_training_instances.csv")

Prepare the first person level data

In [3]:
fp_labels = []
fp_index_in_df = []
# index = 

fp_0 = "'first_person': 0"
fp_1 = "'first_person': 1"

# the actual extraction
for i in range(len(df)):
    if fp_0 in df.labels[i] or fp_1 in df.labels[i]:
        fp_index_in_df.append(1)
        if df.labels[i].find(fp_0) != -1:
            fp_labels.append(df.labels[i][df.labels[i].find(fp_0) + len(fp_0) - 1])
        else:
            fp_labels.append(df.labels[i][df.labels[i].find(fp_1) + len(fp_0) - 1])
    else:
        fp_index_in_df.append(0)
fp_labels = list(map(int, fp_labels))
print(fp_labels[:20])
# print(subset_vec_alc_fp[:10])
print('length of first person level labels: ', len(fp_labels)) # 9816
print(fp_index_in_df[:20])
print('length of first person level index in df: ', len(fp_index_in_df)) # 15651
index_fp = list(np.where(fp_index_in_df)[0])
print(index_fp[:15])
print('length of first person level df index: ', len(index_fp)) # 9816
df_fp = df.iloc[index_fp,]
# df_fp.head
# fp_labels = np.asarray(fp_labels)

# train test dataset split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_fp, fp_labels, test_size=0.2, random_state=26)

[0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1]
length of first person level labels:  9816
[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
length of first person level index in df:  15651
[1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15, 16, 17]
length of first person level df index:  9816


First we check the initial model performance

In [4]:
%%time
clf_fpa_initial.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Wall time: 5min 30s


Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=F...probability=True,
  random_state=None, shrinking=True, tol=0.0008753898561476732,
  verbose=False))])

In [5]:
# print(type(y_test))
# print(y_test[:10])
# # computeAccuracy(y_fpa_initial, y_test)
# auc_score_initial = computeAUC(clf_fpa_initial, X_test, y_test, plot=False)

In [6]:
# predict in testing set.
y_fpa_initial = clf_fpa_initial.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_fpa_initial, y_test)
f1_initial = computeF1Score(y_test, y_fpa_initial)
print('F1 Score: ', f1_initial)
auc_score_initial = computeAUC(clf_fpa_initial, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_initial)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  1964
pretrained alcohol classifer:  626 different item in whole dataset
accuracy: 68.126273%
F1 Score:  0.7904953145917002
alcohol LR AUC:  0.7374843994644761


Then we make our own RF model on first person level

In [9]:
# RF
clf_fp_RF = AlcoholPipeline(global_features=["text"]).pipeline(RandomForestClassifier())
clf_fp_RF.steps

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
      ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              oob_score=False, random_state=None, verbose=0,
              warm_start=False))]

In [18]:
clf_fp_RF.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

Predict in testing set

In [20]:
# predict in testing set.
y_fp_RF = clf_fp_RF.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_fp_RF, y_test)
f1_RF = computeF1Score(y_test, y_fp_RF)
print('F1 Score: ', f1_RF)
auc_score_RF = computeAUC(clf_fp_RF, X_test, y_test, plot=False, RF=True)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol RF AUC: ', auc_score_RF)

length of total comps:  1964
pretrained alcohol classifer:  759 different item in whole dataset
accuracy: 61.354379%
F1 Score:  0.7112970711297072
alcohol RF AUC:  0.6140535296920739


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Test in training set to figure out why the test accuracy is low

In [19]:
# test in training set to figure out why the test accuracy is low
y_train_fp_RF = clf_fp_RF.predict(X_train)
computeAccuracy(y_train_fp_RF, y_train)
f1_score_RF_train = computeF1Score(y_train, y_train_fp_RF)
print('F1 Score: ', f1_score_RF_train)
auc_score_RF_train = computeAUC(clf_fp_RF, X_train, y_train, plot=False, RF=True)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol RF AUC: ', auc_score_RF_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  7852
pretrained alcohol classifer:  75 different item in whole dataset
accuracy: 99.044829%
F1 Score:  0.9926231926822072
alcohol RF AUC:  0.9992878702617849


We can see the performance in training set is really good. ??

Update structure parameters in RF model. To make sure they become same with the initial model

In [21]:
clf_fp_RF_updateParams = AlcoholPipeline(global_features=["text"]).pipeline(RandomForestClassifier())
params_RF = {'features__text__tfidf__dtype':np.int64, 'features__text__tfidf__max_features':86124, 'features__text__tfidf__ngram_range':(1,3) 
            }
# to see all parameters in model
# clf_alc_LR_updateParams.get_params().keys()
clf_fp_RF_updateParams.set_params(**params_RF)
clf_fp_RF_updateParams.steps

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=86124, min_df=1,
     ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              oob_score=False, random_state=None, verbose=0,
              warm_start=False))]

Train the LR model with updated parameters, and predict in testing set

In [22]:
clf_fp_RF_updateParams.fit(X_train, y_train)
# predict in testing set.
y_fp_RF_updateParams = clf_fp_RF_updateParams.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_fp_RF_updateParams, y_test)
f1_score_RF_updateParams = computeF1Score(y_test, y_fp_RF_updateParams)
print('F1 Score: ', f1_score_RF_updateParams)
auc_score_RF_updateParams = computeAUC(clf_fp_RF_updateParams, X_test, y_test, plot=False, RF=True)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol RF AUC: ', auc_score_RF_updateParams)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  1964
pretrained alcohol classifer:  684 different item in whole dataset
accuracy: 65.173116%
F1 Score:  0.745156482861401
alcohol RF AUC:  0.66016360707073


Test in training set to figure out why the test accuracy is low

In [24]:
# test in training set to figure out why the test accuracy is low
y_train_fp_RF_updateParams = clf_fp_RF_updateParams.predict(X_train)
computeAccuracy(y_train_fp_RF_updateParams, y_train)
f1_score_RF_train_updateParams = computeF1Score(y_train, y_train_fp_RF_updateParams)
print('F1 Score: ', f1_score_RF_train_updateParams)
auc_score_RF_train_updateParams = computeAUC(clf_fp_RF_updateParams, X_train, y_train, plot=False, RF=True)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol RF AUC: ', auc_score_RF_train_updateParams)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  7852
pretrained alcohol classifer:  73 different item in whole dataset
accuracy: 99.070301%
F1 Score:  0.9928396272682687
alcohol RF AUC:  0.9990868828140802


The performance in training set is really good (almost 1). However, the performance in test set is not so well. Besides, the improvement after parameters updated (from initial model) is small. Maybe grid search and cross validation may help improve the performance